# Tackle Data Drift with BigQueryML 
> A practical & easy method to detect when you have data issues in your machine learning pipeline.  

Sometimes NULLs or other nuances creep into your inference data. Some call it "data drift" in the machine learning industry. I think of it as, "Did I have any data changes or issues upstream before I make my "real" predictions?" Below you will learn how to check whether there are any data issues by **building a simple model to predict whether a record is part of your "train" or "test-inference" dataset**. If a simple model can figure it out, then you have 
an issue! 

![](https://github.com/david-dirring/data-wrangler-in-ml-world/blob/master/_notebooks/my_icons/No_More_Just_Hoping.png?raw=1)


I have been working on a really fun binary classification model at work.  We have a training process that runs weekly, but our batch prediction process (let's call it "test-time inference" process like Rachel Thomas does [here](https://www.fast.ai/2017/11/13/validation-sets/)) runs multiple times per day.  Much of the feature data changes intraday, **so I have to be sure that the test-time inference data that we feed into our weekly trained model (a pickle file) has the same charateristics as the data that it trained on.**  I have developed a script in BigQuery to use [BQML](https://cloud.google.com/bigquery-ml/docs/reference) that runs every time my test-time inference process runs to check that my "training" data looks similar enough to my "test-time inference" data.

**LEFT SIDE OF IMAGE BELOW:** When we learn about machine learning, we typically do the left side of the figure below. We get the data, we add features, we train, we hold out on some data, and see how we would do on that holdout set. What we don't learn about is how you should SAVE down that model file somewhere so that your "test-inference" process can pick it up later. We use GCP at work to store the files, so we save it down to a Google Cloud Storage bucket (equivalent of S3 at AWS).

**RIGHT SIDE OF IMAGE BELOW**: Before I grab the model file and make predictions on my test-inference data, I need to be sure that this data looks similar enough to the data that the model trained on. If you do not check this, you run the risk of decreasing your accuracy. I will explain how I do this below in BigQuery.


![](https://github.com/david-dirring/data-wrangler-in-ml-world/blob/master/_notebooks/my_icons/no_more_just_hoping_data_is_right_training_and_inference.png?raw=1)

## Simple BQML Script & Model to Alert You To Data Issues

I have been wrangling data and data storytelling for over 13 years. 
I love BigQuery. I love how fast & powerful it is. I also love the work they are putting into BQML. 

For this demonstration, I will pull in a dataset from a BigQuery public dataset -- Daily Liquor Sales in Iowa. I had a friend in who lived in Iowa. I visited there a few times and was shocked when I saw liquor in a grocery store! 

The script below will roughly follow these steps:

1.   **Build** a table 
2.   **Split** table into chunks for train / test / validation / test-inference. In real life, you'll have a process to update the test-inference dataset. The "train / test / validation" is the left side of image above. The "test-inference" is right side.
3.   **Grab 10k records** from train and from test-inference.
4.   Build simple **BQML model** to predict which records are from "train" (0) or from "test-inference" (1)
5.   Look at AUC of simple model to determine if there are data issues. **Fail query if AUC is above your threshold**. I have found .6 or .65 work as a good threshold. AUC of .5 is basically completely random guesses, which implies that the model cannot discern which records are train and which are test-inference.
6.   If issue, then **return the features** that are contributing to the data 
issue.

